In [3]:
import sqlite3
import pandas as pd
import math

from tqdm import tqdm
from joblib import dump, load

models = ['puv', 'pmv', 'pmv_multi', 'ecod', 'abod', 'copod', 'dif']

### Consolidate Results

In [26]:
%%time
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    cur.execute('drop table if exists consolidated_results')

    cur.execute("""
    create table consolidated_results as
        with
        prophet_uv as (
            select
                ds as ts
                ,pred
            from prophet_results_agg
        ),

        prophet_mv as (
            select
                ds as ts
                ,pred
            from prophet_results
            where col = 'reservoirs_mv'
        ),

        prophet_mv_multi as (
            select
                ds as ts
                ,pred
            from prophet_results_mv_multi_agg
        )

        select
            t.ts
            ,t.failure_id
            ,t.day_id
            ,t.pseudo_label
            ,t.pseudo_label_lps

            ,puv.pred as puv_pred
            ,pmv.pred as pmv_pred
            ,pmvm.pred as pmv_multi_pred
            
            ,case 
                when ecod.confidence > 0.5 then ecod.pred 
                else 0
            end as ecod_pred
            ,ecod.pred as ecod_conf

            ,case 
                when abod.confidence > 0.5 then abod.pred 
                else 0
            end as abod_pred
            ,abod.pred as abod_conf

            ,case 
                when copod.confidence > 0.5 then copod.pred 
                else 0
            end as copod_pred
            ,copod.pred as copod_conf

            ,case 
                when dif.confidence > 0.5 then dif.pred 
                else 0
            end as dif_pred
            ,dif.pred as dif_conf

        from train_data as t

            left join prophet_uv as puv
                on puv.ts = t.ts

            left join prophet_mv as pmv
                on pmv.ts = t.ts

            left join prophet_mv_multi as pmvm
                on pmvm.ts = t.ts

            left join pyod_results as ecod
                on  ecod.ts = t.ts
                and ecod.model = 'ecod'

            left join pyod_results as abod
                on  abod.ts = t.ts
                and abod.model = 'abod'

            left join pyod_results as copod
                on  copod.ts = t.ts
                and copod.model = 'copod'

            left join pyod_results as dif
                on  dif.ts = t.ts
                and dif.model = 'dif'

        where
            date(t.ts) <= '2022-03-02'
    """)

CPU times: user 29.7 s, sys: 5.07 s, total: 34.8 s
Wall time: 41.7 s


### Calculate Pseudo-Metrics

In [31]:
with sqlite3.connect('./data/data.db') as con:

    # Dynamically generate SQL
    sql_str = """
        select
            count(*) as n_rows
            ,sum(pseudo_label) as n_outliers
            ,sum(case when pseudo_label is null then 1 else 0 end) as n_inliers
    """

    for model in models:
        sql_str += f"""
            ,sum({model}_pred) as {model}_total_outliers
            ,sum(case when {model}_pred = 0 then 1 else 0 end) as {model}_total_inliers
            ,sum(case when pseudo_label is not null then {model}_pred else 0 end) as {model}_TP
            ,sum(case when pseudo_label is null then {model}_pred else 0 end) as {model}_FP
            ,sum(case when pseudo_label is null and {model}_pred = 0 then 1 else 0 end) as {model}_TN
            ,sum(case when pseudo_label is not null and {model}_pred = 0 then 1 else 0 end) as {model}_FN
        """

    sql_str += """
        from consolidated_results
        where failure_id is null
    """

    # Execute
    df = pd.read_sql(sql_str, con=con)

In [32]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(df.transpose())

,0
n_rows,4368986
n_outliers,504006
n_inliers,3864980
puv_total_outliers,134998
puv_total_inliers,4233988
puv_TP,13841
puv_FP,121157
puv_TN,3743823
puv_FN,490165
pmv_total_outliers,599047


In [33]:
# Calculate model metrics for imbalanced classification
results = df.to_dict(orient='index')[0]
metrics = []

for model in models:
    tp, fp, tn, fn = results[f'{model}_TP'], results[f'{model}_FP'], results[f'{model}_TN'], results[f'{model}_FN']
    n_outliers = results['n_outliers']
    n_inliers = results['n_inliers']

    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*precision*recall)/(precision + recall)

    percent_inliers = fp / n_inliers
    percent_outliers = tp / n_outliers

    metrics.append([model, accuracy, precision, recall, f1, percent_inliers, percent_outliers, tp, fp, tn, fn])

In [34]:
results_df = pd.DataFrame(metrics, columns=['model', 'accuracy', 'precision', 'recall', 'f1', '% inliers', '% outliers', 'tp', 'fp', 'tn', 'fn'])

with pd.option_context('display.float_format', '{:.2%}'.format):
    display(results_df.sort_values(by='precision', ascending=False))

,model,accuracy,precision,recall,f1,% inliers,% outliers,tp,fp,tn,fn
4,abod,11.54%,11.54%,99.99%,20.68%,100.00%,99.99%,503973,3864931,49,33
2,pmv_multi,87.56%,10.50%,1.04%,1.90%,1.16%,1.04%,5254,44786,3820194,498752
0,puv,86.01%,10.25%,2.75%,4.33%,3.13%,2.75%,13841,121157,3743823,490165
6,dif,87.26%,7.32%,0.89%,1.59%,1.47%,0.89%,4500,56983,3807997,499506
3,ecod,87.17%,6.06%,0.77%,1.37%,1.57%,0.77%,3905,60565,3804415,500101
5,copod,87.46%,5.99%,0.59%,1.07%,1.21%,0.59%,2973,46699,3818281,501033
1,pmv,75.85%,4.00%,4.76%,4.35%,14.88%,4.76%,23988,575059,3289921,480018


In [35]:
df[[col for col in df.columns if 'puv' in col]].transpose()

,0
puv_total_outliers,134998
puv_total_inliers,4233988
puv_TP,13841
puv_FP,121157
puv_TN,3743823
puv_FN,490165


### Calculate Event Durations

In [12]:
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    cur.execute('drop table if exists outlier_events')
    cur.execute("""
        create table outlier_events (
            model text,
            event_id int,
            start_ts timestamp,
            end_ts timestamp,
            event_duration_in_secs int
        )
    """)
    cur.execute("""
        insert into outlier_events (model, event_id, start_ts, end_ts, event_duration_in_secs)
        values 
            ('pseudo_label', 1, '2022-02-21 06:00:00', '2022-02-28 02:00:00', 504006), 
            ('failure', 1, '2022-02-28 21:53:00', '2022-03-01 02:00:00', 14820)
    """)

    for model in tqdm(models):
        cur.execute(f"""
            with 
            lagged as (
                select
                    ts
                    ,{model}_pred
                    ,lag({model}_pred, 1, 0) over (order by ts asc) as prev_{model}_pred
                from consolidated_results
            ),
            flagged as (
                select
                    ts
                    ,{model}_pred
                    ,sum(case when {model}_pred != prev_{model}_pred then 1 else 0 end) over (order by ts rows unbounded preceding) as event_id
                from lagged
            )
            insert into outlier_events (model, event_id, start_ts, end_ts, event_duration_in_secs)
            select
                '{model}' as model
                ,event_id
                ,min(ts) as start_ts
                ,max(ts) as end_ts
                ,count(*) as event_duration_in_secs
            from flagged
            where {model}_pred = 1
            group by event_id
            order by start_ts asc
        """)

100%|██████████| 7/7 [00:56<00:00,  8.06s/it]


In [13]:
# Taken from: https://stackoverflow.com/questions/2298339/standard-deviation-for-sqlite
class StdevFunc:
    def __init__(self):
        self.M = 0.0
        self.S = 0.0
        self.k = 1

    def step(self, value):
        if value is None:
            return
        tM = self.M
        self.M += (value - tM) / self.k
        self.S += (value - tM) * (value - self.M)
        self.k += 1

    def finalize(self):
        if self.k < 3:
            return None
        return math.sqrt(self.S / (self.k-2))


with sqlite3.connect('./data/data.db') as con:
    con.create_aggregate("stdev", 1, StdevFunc)

    events = pd.read_sql("""
        select
            model
            ,count(*) as n_events
            ,sum(case when event_duration_in_secs <= 60 then 1 else 0 end) as n_extra_short_events
            ,sum(case when event_duration_in_secs <= 60*5 then 1 else 0 end) as n_short_events
            ,sum(case when event_duration_in_secs > 60*5  and event_duration_in_secs <= 60*60 then 1 else 0 end) as n_medium_events
            ,sum(case when event_duration_in_secs > 60*60 and event_duration_in_secs <= 60*60*8 then 1 else 0 end) as n_long_events
            ,sum(case when event_duration_in_secs > 60*60*8 then 1 else 0 end) as n_extra_long_events
            ,min(event_duration_in_secs) as min_event_duration
            ,max(event_duration_in_secs) as max_event_duration
            ,avg(event_duration_in_secs) as avg_event_duration
            ,stdev(event_duration_in_secs) as std_event_duration
                            
        from outlier_events
        group by model
        order by n_events desc
    """, con=con)

In [14]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(events)

,model,n_events,n_extra_short_events,n_short_events,n_medium_events,n_long_events,n_extra_long_events,min_event_duration,max_event_duration,avg_event_duration,std_event_duration
0,pmv,27205,26492,26933,252,20,0,1,23446,22,370
1,pmv_multi,16707,16629,16701,5,1,0,1,4749,4,47
2,puv,12591,12413,12584,6,1,0,1,7126,11,67
3,dif,8817,8720,8790,25,2,0,1,21524,8,244
4,ecod,8233,8074,8210,21,2,0,1,11599,9,156
5,copod,7590,7364,7572,18,0,0,1,729,7,27
6,abod,81,36,41,10,9,21,1,945483,54120,144509
7,pseudo_label,1,0,0,0,0,1,504006,504006,504006,NaN
8,failure,1,0,0,0,1,0,14820,14820,14820,NaN


### Calculate Pseudo-Metrics (Removing Short Events)

In [27]:
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    for model in tqdm(models):
        cur.execute(f"""
            alter table consolidated_results
            add column {model}_event_id int;
        """)

100%|██████████| 7/7 [00:00<00:00, 1484.33it/s]


In [ ]:
%%time
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    for model in tqdm(models):
        cur.execute(f"""
            with 
            lagged as (
                select
                    ts
                    ,{model}_pred
                    ,lag({model}_pred, 1, 0) over (order by ts asc) as prev_{model}_pred
                from consolidated_results
            ),
            flagged as (
                select
                    ts
                    ,{model}_pred
                    ,sum(case when {model}_pred != prev_{model}_pred then 1 else 0 end) over (order by ts rows unbounded preceding) as event_id
                from lagged
            )
            update consolidated_results as r
            set {model}_event_id = f.event_id
            from flagged as f
            where f.ts = r.ts
        """)

100%|██████████| 7/7 [01:46<00:00, 15.26s/it]

CPU times: user 1min 26s, sys: 18 s, total: 1min 44s
Wall time: 1min 46s


In [ ]:
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    for model in tqdm(models):
        cur.execute(f"""
            alter table consolidated_results
            add column {model}_pred_filtered int;
        """)

100%|██████████| 7/7 [00:00<00:00, 102.88it/s]


In [30]:
with sqlite3.connect('./data/data.db') as con:
    cur = con.cursor()

    for model in tqdm(models):

        cur.execute(f"""
            update consolidated_results
            set {model}_pred_filtered = null
        """)

        cur.execute(f"""
            update consolidated_results
            set {model}_pred_filtered = {model}_pred
            where {model}_event_id in (
                select event_id
                from outlier_events
                where 
                    model = '{model}'
                    and event_duration_in_secs > 60*5
            )
        """)

        cur.execute(f"""
            update consolidated_results
            set {model}_pred_filtered = 0
            where {model}_pred_filtered is null
        """)

100%|██████████| 7/7 [00:35<00:00,  5.01s/it]


In [10]:
with sqlite3.connect('./data/data.db') as con:

    # Dynamically generate SQL
    sql_str = """
        select
            count(*) as n_rows
            ,sum(pseudo_label) as n_outliers
            ,sum(case when pseudo_label is null then 1 else 0 end) as n_inliers
    """

    for model in models:
        sql_str += f"""
            ,sum({model}_pred_filtered) as {model}_total_outliers
            ,sum(case when {model}_pred_filtered = 0 then 1 else 0 end) as {model}_total_inliers
            ,sum(case when pseudo_label is not null then {model}_pred_filtered else 0 end) as {model}_TP
            ,sum(case when pseudo_label is null then {model}_pred_filtered else 0 end) as {model}_FP
            ,sum(case when pseudo_label is null and {model}_pred_filtered = 0 then 1 else 0 end) as {model}_TN
            ,sum(case when pseudo_label is not null and {model}_pred_filtered = 0 then 1 else 0 end) as {model}_FN
        """

    sql_str += """
        from consolidated_results
        where failure_id is null
    """

    # Execute
    df = pd.read_sql(sql_str, con=con)

In [11]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(df.transpose())

,0
n_rows,4368986
n_outliers,504006
n_inliers,3864980
puv_total_outliers,2759
puv_total_inliers,4366227
puv_TP,449
puv_FP,2310
puv_TN,3862670
puv_FN,503557
pmv_total_outliers,437763


In [19]:
# Calculate model metrics for imbalanced classification
results = df.to_dict(orient='index')[0]
metrics = []

for model in models:
    tp, fp, tn, fn = results[f'{model}_TP'], results[f'{model}_FP'], results[f'{model}_TN'], results[f'{model}_FN']

    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*precision*recall)/(precision + recall)

    percent_inliers = fp / (fp + tn)
    percent_outliers = tp / (tp + fn)

    ratio_tp_fp = tp / fp

    metrics.append([model, accuracy, precision, recall, f1, percent_inliers, percent_outliers, ratio_tp_fp, tp, fp, tn, fn])

In [20]:
results_df = pd.DataFrame(metrics, columns=['model', 'accuracy', 'precision', 'recall', 'f1', '% inliers +ve', '% outliers +ve', 'ratio tp/fp', 'tp', 'fp', 'tn', 'fn'])

with pd.option_context('display.float_format', '{:.2%}'.format):
    display(results_df.sort_values(by='precision', ascending=False))

,model,accuracy,precision,recall,f1,% inliers +ve,% outliers +ve,ratio tp/fp,tp,fp,tn,fn
0,puv,88.42%,16.27%,0.09%,0.18%,0.06%,0.09%,19.44%,449,2310,3862670,503557
4,abod,11.54%,11.53%,99.95%,20.68%,99.99%,99.95%,13.04%,503747,3864551,429,259
5,copod,88.35%,10.37%,0.13%,0.26%,0.15%,0.13%,11.56%,655,5664,3859316,503351
2,pmv_multi,88.36%,6.72%,0.07%,0.15%,0.13%,0.07%,7.21%,370,5132,3859848,503636
6,dif,87.75%,5.97%,0.42%,0.79%,0.87%,0.42%,6.35%,2129,33507,3831473,501877
3,ecod,88.03%,3.20%,0.13%,0.25%,0.51%,0.13%,3.31%,655,19809,3845171,503351
1,pmv,78.88%,2.18%,1.89%,2.03%,11.08%,1.89%,2.23%,9536,428227,3436753,494470


### LPS Alarm Analysis

In [57]:
with sqlite3.connect('./data/data.db') as con:

    # Dynamically generate SQL
    sql_str = """
        select
            count(*) as n_rows
            ,sum(pseudo_label_lps) as n_outliers
            ,sum(case when pseudo_label_lps is null then 1 else 0 end) as n_inliers
    """

    for model in models:
        sql_str += f"""
            ,sum({model}_pred) as {model}_total_outliers
            ,sum(case when {model}_pred = 0 then 1 else 0 end) as {model}_total_inliers
            ,sum(case when pseudo_label_lps is not null then {model}_pred else 0 end) as {model}_TP
            ,sum(case when pseudo_label_lps is null then {model}_pred else 0 end) as {model}_FP
            ,sum(case when pseudo_label_lps is null and {model}_pred = 0 then 1 else 0 end) as {model}_TN
            ,sum(case when pseudo_label_lps is not null and {model}_pred = 0 then 1 else 0 end) as {model}_FN
        """

    sql_str += """
        from consolidated_results
        where failure_id is null
    """

    # Execute
    df = pd.read_sql(sql_str, con=con)

In [58]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(df.transpose())

,0
n_rows,4368986
n_outliers,723426
n_inliers,3645560
puv_total_outliers,134998
puv_total_inliers,4233988
puv_TP,18213
puv_FP,116785
puv_TN,3528775
puv_FN,705213
pmv_total_outliers,599047


In [68]:
# Calculate model metrics for imbalanced classification
results = df.to_dict(orient='index')[0]
metrics = []

for model in models:
    tp, fp, tn, fn = results[f'{model}_TP'], results[f'{model}_FP'], results[f'{model}_TN'], results[f'{model}_FN']
    n_outliers = results['n_outliers']
    n_inliers = results['n_inliers']

    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*precision*recall)/(precision + recall)

    percent_inliers = fp / n_inliers
    percent_outliers = tp / n_outliers

    metrics.append([model, accuracy, precision, recall, f1, percent_inliers, percent_outliers, tp, fp, tn, fn])

In [69]:
results_df = pd.DataFrame(metrics, columns=['model', 'accuracy', 'precision', 'recall', 'f1', '% inliers', '% outliers', 'tp', 'fp', 'tn', 'fn'])

with pd.option_context('display.float_format', '{:.1%}'.format):
    display(results_df.loc[~results_df.model.isin(['pmv', 'pmv_multi']), :].sort_values(by='precision', ascending=False))

,model,accuracy,precision,recall,f1,% inliers,% outliers,tp,fp,tn,fn
4,abod,16.6%,16.6%,100.0%,28.4%,100.0%,100.0%,723413,3645491,69,13
0,puv,81.2%,13.5%,2.5%,4.2%,3.2%,2.5%,18213,116785,3528775,705213
6,dif,82.2%,7.2%,0.6%,1.1%,1.6%,0.6%,4436,57047,3588513,718990
3,ecod,82.2%,6.4%,0.6%,1.0%,1.7%,0.6%,4102,60368,3585192,719324
5,copod,82.4%,4.6%,0.3%,0.6%,1.3%,0.3%,2265,47407,3598153,721161


### LPS Alarm Analysis - Removing Short Events

In [4]:
with sqlite3.connect('./data/data.db') as con:

    # Dynamically generate SQL
    sql_str = """
        select
            count(*) as n_rows
            ,sum(pseudo_label_lps) as n_outliers
            ,sum(case when pseudo_label_lps is null then 1 else 0 end) as n_inliers
    """

    for model in models:
        sql_str += f"""
            ,sum({model}_pred_filtered) as {model}_total_outliers
            ,sum(case when {model}_pred_filtered = 0 then 1 else 0 end) as {model}_total_inliers
            ,sum(case when pseudo_label_lps is not null then {model}_pred_filtered else 0 end) as {model}_TP
            ,sum(case when pseudo_label_lps is null then {model}_pred_filtered else 0 end) as {model}_FP
            ,sum(case when pseudo_label_lps is null and {model}_pred_filtered = 0 then 1 else 0 end) as {model}_TN
            ,sum(case when pseudo_label_lps is not null and {model}_pred_filtered = 0 then 1 else 0 end) as {model}_FN
        """

    sql_str += """
        from consolidated_results
        where failure_id is null
    """

    # Execute
    df = pd.read_sql(sql_str, con=con)

In [5]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(df.transpose())

,0
n_rows,4368986
n_outliers,723426
n_inliers,3645560
puv_total_outliers,2759
puv_total_inliers,4366227
puv_TP,0
puv_FP,2759
puv_TN,3642801
puv_FN,723426
pmv_total_outliers,437763


In [7]:
# Calculate model metrics for imbalanced classification
results = df.to_dict(orient='index')[0]
metrics = []

for model in models:
    tp, fp, tn, fn = results[f'{model}_TP'], results[f'{model}_FP'], results[f'{model}_TN'], results[f'{model}_FN']
    n_outliers = results['n_outliers']
    n_inliers = results['n_inliers']

    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    percent_inliers = fp / n_inliers
    percent_outliers = tp / n_outliers

    metrics.append([model, accuracy, precision, recall, percent_inliers, percent_outliers, tp, fp, tn, fn])

In [9]:
results_df = pd.DataFrame(metrics, columns=['model', 'accuracy', 'precision', 'recall', '% inliers', '% outliers', 'tp', 'fp', 'tn', 'fn'])

with pd.option_context('display.float_format', '{:.1%}'.format):
    display(results_df.loc[~results_df.model.isin(['pmv', 'pmv_multi']), :].sort_values(by='precision', ascending=False))

,model,accuracy,precision,recall,% inliers,% outliers,tp,fp,tn,fn
4,abod,16.6%,16.6%,100.0%,100.0%,100.0%,723300,3644998,562,126
6,dif,82.7%,2.5%,0.1%,1.0%,0.1%,900,34736,3610824,722526
5,copod,83.3%,0.2%,0.0%,0.2%,0.0%,14,6305,3639255,723412
3,ecod,83.0%,0.1%,0.0%,0.6%,0.0%,13,20451,3625109,723413
0,puv,83.4%,0.0%,0.0%,0.1%,0.0%,0,2759,3642801,723426
